In [ ]:
import brightway2 as bw
import pandas as pd
import numpy as np

In [ ]:
# bw.projects.set_current('GSA for oases') #TODO choose project
bw.projects.set_current('exiobase_reproduce_results')
co = bw.Database('CH consumption 1.0')
average_consump = co.search('ch hh average consumption')[0]
average_consump

In [ ]:
list_functional_units = []
for exc in average_consump.exchanges():
    if exc['type'] != 'production':
        list_functional_units.append({exc['input']: exc['amount']})
        
list_methods = [
    ('IPCC 2013', 'climate change', 'GWP 100a'),
#     ('ReCiPe Endpoint (H,A)', 'resources', 'total'),
#     ('ReCiPe Endpoint (H,A)', 'ecosystem quality', 'total'),
#     ('ReCiPe Endpoint (H,A)', 'human health', 'total'),
#     ('USEtox', 'ecotoxicity', 'total'),
#     ('USEtox', 'human toxicity', 'total'),
]

In [ ]:
%%time
bw.calculation_setups['av_consump_contribution_analysis'] = {'inv':list_functional_units, 'ia':list_methods}
myMultiLCA = bw.MultiLCA('av_consump_contribution_analysis')
lcia_unit = bw.Method(list_methods[0]).metadata['unit']
fus = [bw.get_activity(list(el.keys())[0])['name'][:25] for el in list_functional_units]
df = pd.DataFrame(index=fus, columns=list_methods, data=myMultiLCA.results)
df['exchanges'] = [list(fu.values())[0] for fu in list_functional_units]
df['units'] = [bw.get_activity(list(el.keys())[0])['unit'] for el in list_functional_units]
# df.sort_values(lcia_unit, ascending=False, inplace=True)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(df)

# Biosphere

In [ ]:
import bw2analyzer

In [ ]:
lca = bw.LCA({average_consump: 1}, list_methods[0])

In [ ]:
lca.lci()
lca.lcia()
lca.score

In [ ]:
ca = bw2analyzer.ContributionAnalysis()

In [ ]:
ca.annotated_top_processes(lca)

# Contribution analysis for sectors

In [ ]:
import brightway2 as bw
import pandas as pd
import plotly.graph_objects as go

In [ ]:
%%time
# project = 'GSA for oases'
project = 'exiobase_reproduce_results'
bw.projects.set_current(project)
co = bw.Database("CH consumption 1.0")
demand_act = co.search('ch hh average consumption')
assert len(demand_act) == 1
demand = {demand_act[0]: 1}
list_methods = [
    ('IPCC 2013', 'climate change', 'GWP 100a'),
    ('ReCiPe Endpoint (H,A)', 'resources', 'total'),
    ('ReCiPe Endpoint (H,A)', 'ecosystem quality', 'total'),
    ('ReCiPe Endpoint (H,A)', 'human health', 'total'),
    ('USEtox', 'ecotoxicity', 'total'),
    ('USEtox', 'human toxicity', 'total'),
]
sectors = [act for act in co if 'sector' in act["name"]]
list_fus = []
for act in sectors:
    list_fus.append({act: 1})
    
bw.calculation_setups['sector_contribution_analysis'] = {'inv':list_fus, 'ia':list_methods}
myMultiLCA = bw.MultiLCA('sector_contribution_analysis')
# lcia_unit = bw.Method(list_methods[0]).metadata['unit']
fus = [bw.get_activity(list(el.keys())[0])['name'][:] for el in list_fus]
df = pd.DataFrame(index=fus, columns=list_methods, data=myMultiLCA.results)
df['units'] = [bw.get_activity(list(el.keys())[0])['unit'] for el in list_fus]
# df.sort_values(lcia_unit, ascending=False, inplace=True)

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(df)

In [ ]:
df_plot = df.iloc[:7]
y = df_plot.index.values
y = [el.replace('sector','') for el in y]
y[1] = 'Housing, water, electricity, gas'
y[3] = 'Durable goods (computers, printers)'
x = df_plot['kg CO2-Eq'].values

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Bar(x=x,y=y,orientation='h',)
)
fig.update_layout(
    height=200,
    width=450,
    margin=dict(t=0,b=0,l=250,r=0),
    yaxis=dict(autorange="reversed"),
    xaxis_title='LCIA scores, [kg CO2-eq]'
)


In [ ]:
fig.write_image('contribution_analysis_esd.pdf')

In [ ]:
# import brightway2 as bw
# bw.projects.set_current("GSA for oases")
# co = bw.Database("CH consumption 1.0")
# demand_act = co.search('ch hh average consumption')
# assert len(demand_act) == 1
# demand = {demand_act[0]: 1}
# list_methods = [
#     ('IPCC 2013', 'climate change', 'GWP 100a'),
# ]
# lca = bw.LCA(demand, list_methods[0])
# lca.lci()
# lca.lcia()
# lca.score

# Maritime shipping for Christian

In [1]:
import brightway2 as bw
import numpy as np
from pypardiso import spsolve

In [2]:
bw.projects.set_current("GSA for paper")

In [3]:
ei = bw.Database('ecoinvent 3.6 cutoff')
co = bw.Database("CH consumption 1.0")
methods = [('IPCC 2013', 'climate change', 'GWP 100a')]
co = bw.Database("CH consumption 1.0")
demand_act = co.search('ch hh average consumption')
assert len(demand_act) == 1
demand = {demand_act[0]: 1}

lca = bw.LCA(demand, methods[0])
lca.lci()
lca.lcia()
reverse_dict = lca.reverse_dict()
lca.score

1143.4935851845898

In [4]:
tag = "freight, sea"

maritime_acts_cols = {
    act: lca.activity_dict[( act['database'], act['code'] )] for act in ei if tag in act['name'] 
}
Af = lca.characterization_matrix * lca.biosphere_matrix * \
     np.diag(spsolve(lca.technosphere_matrix, lca.demand_array))
score_array = np.sum(Af, axis=0)
direct_scores = {k:score_array[v] for k,v in maritime_acts_cols.items()}

In [5]:
def get_new_fus(old_fus):
    new_fus = {}
    k = 0
    for fu in old_fus:
        act = list(fu.keys())[0]
        act_amount = list(fu.values())[0]
        for exc in act.exchanges():
            if exc['type'] == 'technosphere':
                amount_prev = new_fus.get(exc.input, 0)
                new_fus[exc.input] = amount_prev + act_amount*exc['amount']
    return [{k:v} for k,v in new_fus.items()]

In [6]:
mar_list = [act for act in ei if tag in act['name']]

mar_dict = {}
for i,m in enumerate(mar_list):
    all_excs = list(m.exchanges())
    excs = []
    for exc in all_excs:
        if tag in exc.input['name']:
            excs.append({exc.input: exc.amount})
    mar_dict[i] = {
        "output": m,
        "inputs": excs,
        "length": len(excs)
    }

In [21]:
mar_list

['transport, freight, sea, tanker for petroleum' (ton kilometer, GLO, None),
 'transport, freight, sea, container ship with reefer, freezing' (ton kilometer, GLO, None),
 'transport, freight, sea, container ship with reefer, cooling' (ton kilometer, GLO, None),
 'market for transport, freight, sea, container ship' (ton kilometer, GLO, None),
 'market for transport, freight, sea, ferry' (ton kilometer, GLO, None),
 'market for transport, freight, sea, tanker for liquid goods other than petroleum and liquefied natural gas' (ton kilometer, GLO, None),
 'transport, freight, sea, tanker for liquefied natural gas' (ton kilometer, GLO, None),
 'transport, freight, sea, bulk carrier for dry goods' (ton kilometer, GLO, None),
 'transport, freight, sea, tanker for liquid goods other than petroleum and liquefied natural gas' (ton kilometer, GLO, None),
 'transport, freight, sea, container ship' (ton kilometer, GLO, None),
 'market for transport, freight, sea, tanker for petroleum' (ton kilometer,

In [7]:
max_level = 10
list_functional_units = {1: [demand]}
for level in range(2, max_level+1):
    acts = get_new_fus(list_functional_units[level-1])
    list_functional_units[level] = acts

In [8]:
list_maritime_fus = {}
mask = {}
for level in range(2, max_level+1):
    acts = list_functional_units[level]
    mask[level] = np.array([i for i,a in enumerate(acts) if 'freight, sea' in str(a)])
    list_maritime_fus[level] = []
    for m in mask[level]:
        list_maritime_fus[level].append(
            list_functional_units[level][m]
        )

In [9]:
scores = {}
for level, fus in list_maritime_fus.items():
    if len(fus)>0:
        # Run LCAs
        bw.calculation_setups['contribution_analysis'] = {'inv':fus, 'ia':methods}
        myMultiLCA = bw.MultiLCA('contribution_analysis')
        # Save results
        list_fu_acts = [list(fu_act.keys())[0] for fu_act in fus]
        scores_dict = {fu: 0 for fu in list_fu_acts}
        for i,fu in enumerate(list_fu_acts):
            scores_dict[fu] += myMultiLCA.results[i,:][0]
        scores[level] = scores_dict

In [40]:
list(mar_dict[1]['output'].exchanges())

[Exchange: 1.6025641025641 kg*day 'market for operation, reefer, freezing' (kg*day, GLO, None) to 'transport, freight, sea, container ship with reefer, freezing' (ton kilometer, GLO, None)>,
 Exchange: 1.0 ton kilometer 'market for transport, freight, sea, container ship' (ton kilometer, GLO, None) to 'transport, freight, sea, container ship with reefer, freezing' (ton kilometer, GLO, None)>,
 Exchange: 1.0 ton kilometer 'transport, freight, sea, container ship with reefer, freezing' (ton kilometer, GLO, None) to 'transport, freight, sea, container ship with reefer, freezing' (ton kilometer, GLO, None)>]

In [41]:
mar_dict

{0: {'output': 'transport, freight, sea, tanker for petroleum' (ton kilometer, GLO, None),
  'inputs': [{'transport, freight, sea, tanker for petroleum' (ton kilometer, GLO, None): 1.0}],
  'length': 1},
 1: {'output': 'transport, freight, sea, container ship with reefer, freezing' (ton kilometer, GLO, None),
  'inputs': [{'market for transport, freight, sea, container ship' (ton kilometer, GLO, None): 1.0},
   {'transport, freight, sea, container ship with reefer, freezing' (ton kilometer, GLO, None): 1.0}],
  'length': 2},
 2: {'output': 'transport, freight, sea, container ship with reefer, cooling' (ton kilometer, GLO, None),
  'inputs': [{'market for transport, freight, sea, container ship' (ton kilometer, GLO, None): 1.0},
   {'transport, freight, sea, container ship with reefer, cooling' (ton kilometer, GLO, None): 1.0}],
  'length': 2},
 3: {'output': 'market for transport, freight, sea, container ship' (ton kilometer, GLO, None),
  'inputs': [{'market for transport, freight, se

In [11]:
mar_dict2 = {}
i = 0
for m,col in maritime_acts_cols.items():
    rows = lca.tech_params[lca.tech_params['col']==col]['row']
    acts = []
    for row in rows:
        act = bw.get_activity(lca.reverse_dict()[0][row])
        acts.append(act)
    mar_dict2[i] = {
        "outputs": acts,
        "input": m,
        "length": len(acts)
    }
    i += 1
mar_dict2

{0: {'outputs': ['transport, freight, sea, container ship' (ton kilometer, GLO, None),
   'market group for heavy fuel oil' (kilogram, RER, None),
   'market for heavy fuel oil' (kilogram, CO, None),
   'market for heavy fuel oil' (kilogram, PE, None),
   'market for container ship' (unit, GLO, None),
   'market for heavy fuel oil' (kilogram, RoW, None),
   'market for bilge oil' (kilogram, RoW, None),
   'market for heavy fuel oil' (kilogram, ZA, None),
   'market for heavy fuel oil' (kilogram, BR, None),
   'market for bilge oil' (kilogram, Europe without Switzerland, None),
   'market for bilge oil' (kilogram, CH, None),
   'market for heavy fuel oil' (kilogram, IN, None),
   'market for maintenance, container ship' (unit, GLO, None)],
  'input': 'transport, freight, sea, container ship' (ton kilometer, GLO, None),
  'length': 13},
 1: {'outputs': ['market for transport, freight, sea, container ship with reefer, cooling' (ton kilometer, GLO, None),
   'transport, freight, sea, conta

In [26]:
sum(list(direct_scores.values())) / 1850 * 100

0.34309883951764003

In [42]:
direct_scores

{'transport, freight, sea, container ship' (ton kilometer, GLO, None): 1.0442508751974844,
 'market for transport, freight, sea, container ship with reefer, cooling' (ton kilometer, GLO, None): 0.0,
 'transport, freight, sea, bulk carrier for dry goods' (ton kilometer, GLO, None): 1.913798370779704,
 'market for transport, freight, sea, tanker for liquid goods other than petroleum and liquefied natural gas' (ton kilometer, GLO, None): 0.0,
 'transport, freight, sea, tanker for liquefied natural gas' (ton kilometer, GLO, None): 0.10353608688887686,
 'transport, freight, sea, tanker for petroleum' (ton kilometer, GLO, None): 3.273808981567363,
 'transport, freight, sea, container ship with reefer, cooling' (ton kilometer, GLO, None): 0.0,
 'market for transport, freight, sea, tanker for petroleum' (ton kilometer, GLO, None): 0.0,
 'market for transport, freight, sea, tanker for liquefied natural gas' (ton kilometer, GLO, None): 0.0,
 'market for transport, freight, sea, container ship' (

In [12]:
list_maritime_fus[4]

[{'market for transport, freight, sea, bulk carrier for dry goods' (ton kilometer, GLO, None): 4.6954924663926345},
 {'market for transport, freight, sea, container ship with reefer, cooling' (ton kilometer, GLO, None): 16.82804157998949},
 {'market for transport, freight, sea, container ship' (ton kilometer, GLO, None): 7.161385418843002}]

In [13]:
list_maritime_fus[5]

[{'transport, freight, sea, bulk carrier for dry goods' (ton kilometer, GLO, None): 4.6954924663926345},
 {'market for transport, freight, sea, container ship' (ton kilometer, GLO, None): 1.4366627499183091},
 {'market for transport, freight, sea, bulk carrier for dry goods' (ton kilometer, GLO, None): 0.5858700904178393},
 {'transport, freight, sea, container ship with reefer, cooling' (ton kilometer, GLO, None): 16.82804157998949},
 {'transport, freight, sea, container ship' (ton kilometer, GLO, None): 7.161385418843002},
 {'market for transport, freight, sea, container ship with reefer, cooling' (ton kilometer, GLO, None): 1.3437479619301484},
 {'market for transport, freight, sea, tanker for petroleum' (ton kilometer, GLO, None): 0.0014862821010739537}]

In [14]:
list_maritime_fus[6]

[{'transport, freight, sea, container ship' (ton kilometer, GLO, None): 1.4366627499183091},
 {'market for transport, freight, sea, container ship' (ton kilometer, GLO, None): 33.56879731726995},
 {'market for transport, freight, sea, bulk carrier for dry goods' (ton kilometer, GLO, None): 1.9337562511291584},
 {'transport, freight, sea, bulk carrier for dry goods' (ton kilometer, GLO, None): 0.5858700904178393},
 {'market for transport, freight, sea, tanker for petroleum' (ton kilometer, GLO, None): 0.020472060022276906},
 {'market for transport, freight, sea, container ship with reefer, cooling' (ton kilometer, GLO, None): 19.093083921504082},
 {'market for transport, freight, sea, tanker for liquid goods other than petroleum and liquefied natural gas' (ton kilometer, GLO, None): 0.15628291594007873},
 {'transport, freight, sea, container ship with reefer, cooling' (ton kilometer, GLO, None): 1.3437479619301484},
 {'transport, freight, sea, tanker for petroleum' (ton kilometer, GLO, 

In [15]:
list_maritime_fus[7]

[{'market for transport, freight, sea, tanker for petroleum' (ton kilometer, GLO, None): 63.87090655287512},
 {'market for transport, freight, sea, container ship' (ton kilometer, GLO, None): 3.501573852291081},
 {'market for transport, freight, sea, bulk carrier for dry goods' (ton kilometer, GLO, None): 8.159042939203303},
 {'market for transport, freight, sea, tanker for liquid goods other than petroleum and liquefied natural gas' (ton kilometer, GLO, None): 0.07295530738046224},
 {'transport, freight, sea, container ship' (ton kilometer, GLO, None): 33.56879731726995},
 {'transport, freight, sea, bulk carrier for dry goods' (ton kilometer, GLO, None): 1.9337562511291584},
 {'transport, freight, sea, tanker for petroleum' (ton kilometer, GLO, None): 0.020472060022276906},
 {'transport, freight, sea, container ship with reefer, cooling' (ton kilometer, GLO, None): 19.093083921504082},
 {'transport, freight, sea, tanker for liquid goods other than petroleum and liquefied natural gas' 

In [16]:
list_maritime_fus[8]

[{'market for transport, freight, sea, tanker for petroleum' (ton kilometer, GLO, None): 445.2112861998409},
 {'transport, freight, sea, tanker for petroleum' (ton kilometer, GLO, None): 63.87090655287512},
 {'market for transport, freight, sea, container ship' (ton kilometer, GLO, None): 38.88072491265728},
 {'market for transport, freight, sea, bulk carrier for dry goods' (ton kilometer, GLO, None): 15.85324661709054},
 {'transport, freight, sea, container ship' (ton kilometer, GLO, None): 3.501573852291081},
 {'transport, freight, sea, bulk carrier for dry goods' (ton kilometer, GLO, None): 8.159042939203303},
 {'transport, freight, sea, tanker for liquid goods other than petroleum and liquefied natural gas' (ton kilometer, GLO, None): 0.07295530738046224},
 {'market for transport, freight, sea, tanker for liquid goods other than petroleum and liquefied natural gas' (ton kilometer, GLO, None): 0.205928539113951},
 {'market for transport, freight, sea, container ship with reefer, coo

In [17]:
list_maritime_fus[9]

[{'market for transport, freight, sea, tanker for petroleum' (ton kilometer, GLO, None): 12.254954930519162},
 {'transport, freight, sea, tanker for petroleum' (ton kilometer, GLO, None): 445.2112861998409},
 {'market for transport, freight, sea, bulk carrier for dry goods' (ton kilometer, GLO, None): 9.674244402350153},
 {'market for transport, freight, sea, container ship' (ton kilometer, GLO, None): 3.0970428486356165},
 {'transport, freight, sea, container ship' (ton kilometer, GLO, None): 38.88072491265728},
 {'transport, freight, sea, bulk carrier for dry goods' (ton kilometer, GLO, None): 15.85324661709054},
 {'transport, freight, sea, tanker for liquid goods other than petroleum and liquefied natural gas' (ton kilometer, GLO, None): 0.205928539113951},
 {'market for transport, freight, sea, tanker for liquid goods other than petroleum and liquefied natural gas' (ton kilometer, GLO, None): 0.11071317208797347},
 {'market for transport, freight, sea, tanker for liquefied natural 

In [18]:
list_maritime_fus[10]

[{'market for transport, freight, sea, container ship' (ton kilometer, GLO, None): 8.274243739202578},
 {'market for transport, freight, sea, bulk carrier for dry goods' (ton kilometer, GLO, None): 19.055115538119527},
 {'market for transport, freight, sea, tanker for petroleum' (ton kilometer, GLO, None): 5.803818661648253},
 {'transport, freight, sea, tanker for petroleum' (ton kilometer, GLO, None): 12.254954930519162},
 {'market for transport, freight, sea, tanker for liquid goods other than petroleum and liquefied natural gas' (ton kilometer, GLO, None): 0.20531385244104466},
 {'transport, freight, sea, bulk carrier for dry goods' (ton kilometer, GLO, None): 9.674244402350153},
 {'transport, freight, sea, container ship' (ton kilometer, GLO, None): 3.0970428486356165},
 {'market for transport, freight, sea, tanker for liquefied natural gas' (ton kilometer, GLO, None): 0.24795724679718328},
 {'transport, freight, sea, tanker for liquid goods other than petroleum and liquefied natur

In [37]:
indirect_scores = {}
for level in range(2,max_level+1):
    list_ = list_maritime_fus[level]
    indirect_score = 0
    for el in list_:
        temp = list(el.values())[0]
        indirect_score += temp
    indirect_scores[level] = indirect_score

In [38]:
indirect_scores

{2: 0,
 3: 0,
 4: 28.68491946522513,
 5: 32.0526865495925,
 6: 58.14015955023293,
 7: 130.46204292730704,
 8: 577.2246256721115,
 9: 527.491782359351,
 10: 59.60838593204807}